In [1]:
from selenium.webdriver.common.by import By
from datetime import datetime
import selenium as sel
from tqdm import tqdm
import pandas as pd
import os
import sys
cwd = os.getcwd()
cwd = cwd.replace("Notebooks", "Scripts")
sys.path.insert(0, cwd.replace("\\\\", "\\"))
import base as bs

In [2]:
#Empresa con la cual vamos a extraer los articulos
empresa = input("Digite la empresa a extraer: ").lower().strip()
if " " in empresa:
    empresa_ = empresa.replace(" ", "%20")
else:
    empresa_ = empresa
revista = "elColombiano"

In [3]:
driver = bs.ejecutar_driver(f'https://www.elcolombiano.com/busqueda/-/search/{empresa_}/false/false/19810311/20230311/date/true/true/0/0/meta/0/0/0/1')

In [4]:
# Articulos a extraer
num_paginas = input("Ingrese el numero de páginas que desea recorrer").lower()
url_princ = driver.current_url[:-1]
titulares = []
for i in tqdm(range(1, int(num_paginas) + 1)):
    aux = str(i)
    url_a_buscar = url_princ+aux
    driver.get(url_a_buscar)
    articulos = driver.find_elements(By.XPATH, './/li[@class="element   full-access norestricted"]')
    
    for art in tqdm(articulos):
        url = art.find_element(By.XPATH, './/div[contains(@class, "right")]//a').get_attribute('href')
        if not(bs.existedb(url, "database", empresa)):
            titulo = art.find_element(By.XPATH, './/h3[contains(@class, "titulo-noticia")]//span').text
            fechaP = art.find_element(By.XPATH, './/div[contains(@class, "fecha")]//span').text
            fechaP = datetime.strptime(fechaP, '%d / %m / %Y')
            tema = art.find_element(By.XPATH, './/div[contains(@class, "information")]//a').text
            autor = art.find_element(By.XPATH, './/span[contains(@class, "autor")]').text
            lista_tags = art.find_element(By.XPATH, './/div[contains(@class, "tags-noticia")]//ul')
            tags = list(map(lambda x: x.text, lista_tags.find_elements(By.XPATH, './/li')))
            imagen = bs.obtener_imagen_col(art)
            titulares.append({'Fecha Extraccion':datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                            'Titulo': titulo,
                            'Fecha Publicacion': fechaP,
                            'URL': url,
                            'Tema': tema,
                            'Autor': autor,
                            'Tags': tags,
                            'Imagen': imagen,
                            'Empresa': empresa,
                            'Fuente': 'El Colombiano'
                            })

100%|██████████| 5/5 [01:52<00:00, 22.45s/it]


In [5]:
for tit in tqdm(titulares):
    driver.get(tit["URL"])
    driver.implicitly_wait(10)
    tit["Contenido"] = bs.obtener_contenido_col(driver)
    tit["Resumen"] = bs.obtener_resumen_col(driver)

100%|██████████| 25/25 [14:13<00:00, 34.15s/it]


In [6]:
df = pd.DataFrame(titulares)
# bs.writeData("database", df)

In [8]:
driver.quit()